In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 30.4 MB/s eta 0:00:00


In [ ]:
import re
import rdflib
import urllib.parse
from rdflib import Graph
import pandas as pd
import openai

In [ ]:
api_key= ''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/CS5588_Hand-On/world_news_data/wsj_headlines_20201101_20201130.csv'

In [ ]:
df = pd.read_csv(path)
df

,date,headline,url
0,2020/11/01,The Battleground States That Will Likely Decid...,https://www.wsj.com/articles/the-battleground-...
1,2020/11/01,Pro-Trump Car Rallies Slow Traffic in New York...,https://www.wsj.com/articles/pro-trump-car-ral...
2,2020/11/01,Trump Barnstorms Battleground States; Biden Ta...,https://www.wsj.com/articles/biden-frames-pres...
3,2020/11/01,Three Things in Play in New York’s Elections,https://www.wsj.com/articles/three-things-in-p...
4,2020/11/01,Political Groups Elude Facebook’s Election Con...,https://www.wsj.com/articles/political-groups-...
...,...,...,...
1435,2020/11/30,Powell Says Fed Actions Unlocked $2 Trillion t...,https://www.wsj.com/articles/federal-reserve-e...
1436,2020/11/30,Pandemic Cuts Foot Traffic in New York’s Top B...,https://www.wsj.com/articles/pandemic-cuts-foo...
1437,2020/11/30,OPEC Defers Decision on Extending Output Curbs,https://www.wsj.com/articles/opec-defers-decis...
1438,2020/11/30,Dow Falls but Finishes Best Month in Three Dec...,https://www.wsj.com/articles/global-stock-mark...


In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')

# Set 'date' as the index
df.set_index('date', inplace=True)

# Resample the DataFrame by week.
# This groups the data into weeks based on the 'date' column and aggregates headlines and URLs into lists for each week.
weekly_batches = df.resample('W').agg(list)

# To view the result, you can print the weekly_batches
print(weekly_batches)

                                                     headline  \
date                                                            
2020-11-01  [The Battleground States That Will Likely Deci...   
2020-11-08  [Trump, Biden Pitch Dueling Visions in Final S...   
2020-11-15  [Joe Biden Focused on Coronavirus to Kick Off ...   
2020-11-22  [GameStop’s New Billionaire Investor Pushes fo...   
2020-11-29  [Hong Kong Activist Joshua Wong Faces Jail Aft...   
2020-12-06  [China Complains of U.S. Harassment of Airline...   

                                                          url  
date                                                           
2020-11-01  [https://www.wsj.com/articles/the-battleground...  
2020-11-08  [https://www.wsj.com/articles/trump-biden-pitc...  
2020-11-15  [https://www.wsj.com/articles/joe-biden-focuse...  
2020-11-22  [https://www.wsj.com/articles/gamestops-new-bi...  
2020-11-29  [https://www.wsj.com/articles/hong-kong-activi...  
2020-12-06  [https://www.wsj.co

In [ ]:
# Create a list of DataFrames for each week
weekly_dfs = [group for _, group in df.groupby(pd.Grouper(freq='W'))]

# Example: To access the first week's DataFrame
print(weekly_dfs[3])  # This will print the first week as a DataFrame

weekly_df = weekly_dfs[3]['headline']

                                                     headline  \
date                                                            
2020-11-16  GameStop’s New Billionaire Investor Pushes for...   
2020-11-16   More Lawmakers Fall Ill Amid Covid-19 Resurgence   
2020-11-16  Taylor Swift’s Early Music Catalog Changes Han...   
2020-11-16  HBO’s ‘Industry’ Recap, Episode 2: What’s the ...   
2020-11-16  Trump Asked Top Aides About Options to Strike ...   
...                                                       ...   
2020-11-22  Trump Exits Open Skies Treaty, Moves to Discar...   
2020-11-22  Bain Capital Raises $800 Million for Second Im...   
2020-11-22  Covid Deaths in Europe Are Overwhelmingly Amon...   
2020-11-22  Why Some Tech Workers Leaving Silicon Valley A...   
2020-11-22  Singapore, a City of Skyscrapers and Little La...   

                                                          url  
date                                                           
2020-11-16  https://www.ws

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import spacy

# Download NLTK resources if they are not already available
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Load SpaCy model for lemmatization
nlp = spacy.load('en_core_web_sm')

def preprocess_text(text):
    # Check if text is a list and join it if necessary
    if isinstance(text, list):
        text = ' '.join(text)

    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and perform lemmatization
    tokens = [token.lemma_ for token in nlp(" ".join([word for word in tokens if word not in stop_words]))]

    # Handle empty strings
    if not tokens:  # Check if tokens list is empty after preprocessing
        return " "  # Replace empty list with a space to avoid NaN

    # Join tokens back to string
    return " ".join(tokens)

# Assuming combined_text is a pandas Series (e.g., from a DataFrame)
# If combined_text is a column from a DataFrame like df['column_name'], use:
# combined_text = df['column_name']

# Apply preprocessing to the correct DataFrame (filtered_df)
# Ensure combined_text is a pandas Series (or DataFrame column)
if isinstance(weekly_df, pd.Series):
    df = weekly_df
    clean_text = df.apply(preprocess_text)
else:
    print("combined_text is not a pandas Series. Please check your data.")

combined_text = '\n'.join(clean_text)
combined_text

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


"gamestop ' new billionaire investor push digital sale few store\nlawmaker fall ill amid covid19 resurgence\ntaylor swift ' early music catalog change hand\nhbo ' ' industry ' recap episode 2 ' competition\ntrump ask top aide option strike iran\ngeorgia republicans worry trump feud could hurt key senate runoff\nairbnb reveal potential noncompliance us sanction\nbiden say delayed transition could affect coronavirus response\ntrump ' afghan adio\nairbnb ' recovery may worth bet\nsec chairman jay clayton leave agency end 2020\nshelton face gop opposition feed nomination set close vote\nberkshire invest drugmaker seek covid19 vaccine\nlawsuit michigan center battle mailin vote count\ntesla add sp 500 index\nshopping mall hope find new life fuel ecommerce\nwe draw plan withdraw troop iraq afghanistan\nairbnb turn deep cutback profit ahead ipo\nglobal regulator highlight need shore moneymarket fund\ntrump biden covid vaccine\ncinemark universal usher change movietheater distribution\nstate t

# Knowledge Graph

In [ ]:
import openai

# Set OpenAI API key
openai.api_key = api_key

# Define input news text
input_news = combined_text

# Define the structured format need in prompt
prompt = f"""
"I will provide you with news headlines labeled as INPUT_NEWS. Your task is to extract structured information from it in the form of triplets for constructing a knowledge graph. Each triplet should be form of (h:type, r, o:type), where 'h' stands for the head entity, 'r' for the relationship, and 'o' for tail entity. The 'type' denotes the category of the corresponding entity.

The Entities should be non-generic and can be classified into the following categories:
ORG: Organization other than government or regulatory bodies (e.g. “Apple Inc.”)
ORG/GOV: Government bodies (e.g., “United States Government”)
ORG/REG: Regulatory bodies (e.g., “Federal Reserve”)
Person: Individuals  (e.g., “Elon Musk”)
GPE: Geopolitical entities such as countries, cities, etc (e.g., “Germany”)
COMP: Companies (e.g.,”Google”)
PRODUCT: Products or services (e.g.,”Iphone”)
EVENT: Specific and Material Events (e.g., “Olympic Games”, “Covid-19”)
SECTOR: Company sectors or industries (e.g.,”Technology Sector”)
ECON_INDICATOR: Economic indicators (e.g.,”Inflation rate), numerical value like 10% is not ECON_INDICATOR
FIN_INSTRUMENT: Financial and market instruments (e.g., “Stocks”)

Relationships r between these entities must be represented by one of the following verbs:
Has, Announce, Operate_In, Introduce, Produce, Control, Participate_In, Impact, Positive_Impact_On, Negative_Impact_On, Relate_To, Is_Member_Of, Invests_In, Raise, Decrease

Remember to conduct entity disambiguation, consolidating different phrases or acronyms that refer to the same entity (for instance, “UK Central Bank” “BOE” and “Bank of England” should be unified as “Bank of England”). Simply each entity of the triplet to be no more than three word. Your output should strictly consist of a list of triplets and nothing else. Do NOT include the redundant triplets or triplets with numerical or date entities.
As an example, consider the following news:
‘Apple Inc. is set to introduce the new iPhone 16 in the technology sector this month. The product’s release is likely to positively impact Apple’s stock value.’


Now, could you please apply this process to the following INPUT_NEWS?

{input_news}

"""

# Call the GPT-4 chat completion
response = openai.ChatCompletion.create(
    model="gpt-4",  # Specify the GPT-4 model
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt}],
    # max_tokens=500,  # Set a max token limit
    temperature=0.5  # Adjust temperature for randomness
)

response_text = response['choices'][0]['message']['content']
# Print the response
print(response_text)


[("gamestop", "Invests_In", "digital sale:PRODUCT"), 
("lawmaker", "Fall_Ill_Amid", "covid19:EVENT"), 
("taylor swift", "Change_Hand", "early music catalog:PRODUCT"), 
("hbo", "Produce", "industry:SECTOR"), 
("trump", "Ask", "top aide:Person"), 
("trump", "Negative_Impact_On", "georgia republicans:ORG"), 
("airbnb", "Reveal", "us sanction:ORG/REG"), 
("biden", "Impact", "coronavirus response:EVENT"), 
("berkshire", "Invests_In", "drugmaker:COMP"), 
("tesla", "Add", "sp 500 index:FIN_INSTRUMENT"), 
("shopping mall", "Participate_In", "ecommerce:SECTOR"), 
("airbnb", "Turn", "profit:FIN_INSTRUMENT"), 
("global regulator", "Highlight", "moneymarket fund:FIN_INSTRUMENT"), 
("trump", "Control", "covid vaccine:PRODUCT"), 
("state", "Raise", "tax revenue:ECON_INDICATOR"), 
("education", "Relate_To", "joe biden:Person"), 
("peru congress", "Choose", "lawmaker francisco sagasti:Person"), 
("stock", "Close", "moderna vaccine result:FIN_INSTRUMENT"), 
("pnc", "Highlight", "challenge european bank

# Query Knowledge Graph

In [ ]:
import ast
import re

# Clean the entities by removing the type suffix (e.g., :GPE, :PRODUCT)
def clean_entity(entity):
    # Remove any part after the colon (e.g., ":GPE", ":EVENT")
    return entity.split(": ")[0]  # Keep the main entity without the suffix

# Given response_text (list of triplets in string format)
def parse_triplets(response_text):
    # Step 2: Clean the text
    response_text_cleaned = response_text.strip()

    # Step 3: Replace fancy quotes with normal quotes for proper parsing
    response_text_cleaned = response_text_cleaned.replace('“', '"').replace('”', '"')

    # Step 4: Clean the entity type suffixes (e.g., ":GPE", ":EVENT")
    # This regex will remove any occurrences of ":TYPE" in the string
    response_text_cleaned = re.sub(r':[A-Za-z]+', '', response_text_cleaned)

    # Step 5: Split the string into individual triplet strings, in case the input is malformed
    # We will try to split it based on the pattern of tuple-like elements
    triplet_strings = re.findall(r'\(([^()]+, [^()]+, [^()]+)\)', response_text_cleaned)

    cleaned_triplets = []

    # Process each triplet
    for triplet_str in triplet_strings:
        try:
            # Convert the cleaned triplet string into a tuple
            triplet_tuple = ast.literal_eval(f'({triplet_str})')
            print(f"Parsed triplet: {triplet_tuple}")

            # Clean the triplet by removing entity types from subject/object
            cleaned_triplet = (clean_entity(triplet_tuple[0]), triplet_tuple[1], clean_entity(triplet_tuple[2]))
            cleaned_triplets.append(cleaned_triplet)

        except (ValueError, SyntaxError) as e:
            # If a triplet is malformed or cannot be parsed, print an error and skip it
            print(f"Error parsing triplet '{triplet_str}': {e}")
            continue

    return cleaned_triplets


# Clean triplets from response text
cleaned_triplets = parse_triplets(response_text)

print("Cleaned triplets:", cleaned_triplets)



Parsed triplet: ('GameStop', 'Invests_In', 'Digital Sale')
Parsed triplet: ('Lawmaker', 'Fall_Ill', 'Covid19')
Parsed triplet: ('Taylor Swift', 'Change_Hand', 'Music Catalog')
Parsed triplet: ('HBO', 'Produce', 'Industry')
Parsed triplet: ('Trump', 'Ask', 'Top Aide')
Parsed triplet: ('Trump', 'Negative_Impact_On', 'Georgia Republicans/GOV')
Parsed triplet: ('Airbnb', 'Reveal', 'Noncompliance')
Parsed triplet: ('Biden', 'Say', 'Delayed Transition')
Parsed triplet: ('Trump', 'Control', 'Afghan')
Parsed triplet: ('Airbnb', 'Recover', 'Profit_INSTRUMENT')
Parsed triplet: ('Jay Clayton', 'Leave', 'SEC/REG')
Parsed triplet: ('Shelton', 'Face', 'GOP Opposition/GOV')
Parsed triplet: ('Berkshire', 'Invests_In', 'Drugmaker')
Parsed triplet: ('Lawsuit', 'Center', 'Mail-in Vote Count')
Parsed triplet: ('Tesla', 'Add', 'SP 500 Index_INSTRUMENT')
Parsed triplet: ('Shopping Mall', 'Find', 'New Life')
Parsed triplet: ('We', 'Draw', 'Withdraw Plan')
Parsed triplet: ('Airbnb', 'Turn', 'Deep Cutback')
Pa

In [ ]:
# Create an RDF Graph from the triplets and save it to a file
def create_rdf_from_triplets(triplets):
    # Initialize an RDF graph
    g = rdflib.Graph()

    # Define a base URI for your entities
    base_uri = "http://example.org/"

    # Add each triplet to the RDF graph
    for triplet in triplets:
        subject, predicate, object = triplet

        # URL-encode and create valid URIs
        subject_uri = rdflib.URIRef(base_uri + urllib.parse.quote(subject.replace(" ", "_")))
        predicate_uri = rdflib.URIRef(base_uri + urllib.parse.quote(predicate.replace(" ", "_")))
        object_uri = rdflib.URIRef(base_uri + urllib.parse.quote(object.replace(" ", "_")))

        # Add the triple to the graph
        g.add((subject_uri, predicate_uri, object_uri))

    # Save the RDF graph to a file (e.g., RDF/XML format)
    rdf_file = "knowledge_graph.rdf"
    g.serialize(rdf_file, format="xml")
    print(f"RDF file saved as '{rdf_file}'")




# Parse the response to get the triplets
triplets = cleaned_triplets
# Create RDF and save to a file
create_rdf_from_triplets(triplets)

RDF file saved as 'knowledge_graph.rdf'


In [ ]:
# Initialize the RDF graph
g = Graph()

# Load the RDF file into the graph
rdf_file = "/content/knowledge_graph.rdf"
g.parse(rdf_file, format="xml")  # Load RDF in RDF/XML format

# SPARQL query to retrieve all triplets (subject, predicate, object)
query = """
SELECT ?subject ?predicate ?object
WHERE {
  ?subject ?predicate ?object .
}
"""


# Assuming you've already run the query and fetched results using `g.query(query)`

# Create a list to store the results
data = []

# Run the SPARQL query
results = g.query(query)

# Iterate through the query results and append each row to the list
for row in results:
    # Extract the value after "http://example.org/"
    subject = str(row.subject).split("http://example.org/")[-1]  # Get part after base URI
    predicate = str(row.predicate).split("http://example.org/")[-1]
    obj = str(row.object).split("http://example.org/")[-1]

    data.append({
        'Subject': subject,  # Now only the part after "http://example.org/"
        'Predicate': predicate,
        'Object': obj
    })


data

[{'Subject': 'Azerbaijan',
  'Predicate': 'Shoot',
  'Object': 'Russian_Helicopter'},
 {'Subject': 'Fed/REG', 'Predicate': 'Say', 'Object': 'Coronavirus'},
 {'Subject': 'UP', 'Predicate': 'Lift', 'Object': 'Ban_on_Beards'},
 {'Subject': '%27Toxic_Politics%27',
  'Predicate': 'Review',
  'Object': 'Xi_Jinping%27s_Clean_House'},
 {'Subject': 'American_University', 'Predicate': 'Wake', 'Object': 'Enough'},
 {'Subject': 'Asymptomatic_Covid-19_Case',
  'Predicate': 'Show',
  'Object': 'Need_Wide_Testing'},
 {'Subject': 'Harvest_2020_Election',
  'Predicate': 'Relate_To',
  'Object': 'Harvest'},
 {'Subject': 'Seattle',
  'Predicate': 'Experience',
  'Object': 'New_Theory_Crime'},
 {'Subject': 'Tech_Rally',
  'Predicate': 'Power',
  'Object': 'US_Stock_INSTRUMENT'},
 {'Subject': 'State/GOV',
  'Predicate': 'Impose',
  'Object': 'Flurry_of_Measures'},
 {'Subject': 'SP_500_INSTRUMENT',
  'Predicate': 'Close',
  'Object': 'Week_Record_Vaccine_Optimism'},
 {'Subject': '%27Crown%27_11_Things',
  '

In [ ]:
k = 5
prompt_llm1 = f"I am planning to invest multiple stocks today. Could you please tell me {k} factors from {data} that could affect market performance?"
prompt_llm2 = f"I am planning to invest today. Please identify the top {k} factors that may affect stock market performance from the following news stories: \n\n{data}"

In [ ]:
final_path = '/content/drive/MyDrive/CS5588_Hand-On/world_factors'

In [ ]:
# Call the GPT-4 chat completion
response_llm = openai.ChatCompletion.create(
    model="gpt-4o",  # Specify the GPT-4 model
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt_llm2}],
    #max_tokens=500,  # Set a max token limit
    temperature=0.5  # Adjust temperature for randomness
)

response_llm_txt = response_llm['choices'][0]['message']['content']
# Define the path where you want to save the file
file_path = f"{final_path}/202011_week3.txt"

# Open the file in write mode and save the string to the file
with open(file_path, 'w') as f:
    f.write(response_llm_txt)
# Print the response
print(response_llm_txt)

Based on the news stories you provided, here are the top 5 factors that may affect stock market performance:

1. **Vaccine Developments and Optimism**:
   - Several stories mention vaccine-related optimism, such as the "SP_500_INSTRUMENT" closing at a record due to vaccine optimism and the "Real_Estate_Stock_INSTRUMENT" soaring because of vaccine hope. Additionally, a "Vaccine" is noted to jolt the market, indicating that progress in vaccine development is a significant market driver.

2. **Covid-19 Impact and Restrictions**:
   - The ongoing impact of Covid-19 is evident in stories about record hospitalizations, the surge in cases leading to school closures, and the strain on systems. This continues to be a major factor affecting market sentiment and economic activities.

3. **Federal Reserve and Economic Policies**:
   - The mention of the "Fed/REG" discussing the coronavirus and the "State/GOV" imposing a flurry of measures indicate that monetary and fiscal policies remain crucial. 

In [ ]:
# Call the GPT-4 chat completion
response_llm = openai.ChatCompletion.create(
    model="gpt-4o",  # Specify the GPT-4 model
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": prompt_llm1}],
    #max_tokens=500,  # Set a max token limit
    temperature=0.5  # Adjust temperature for randomness
)

response_llm_txt = response_llm['choices'][0]['message']['content']
# Print the response
print(response_llm_txt)

When considering factors that could affect market performance, it's important to focus on broader economic, political, and social events or trends. From the list provided, here are five factors that could potentially influence market performance:

1. **Drone Startup's Role in Vaccine Delivery**: The statement about a drone startup aiming to carve a role in the delivery of potential COVID-19 vaccines highlights the importance of logistics and technology companies in the healthcare supply chain. Successful integration of such technologies could boost related stocks.

2. **Tech Startup and H1B Visa Rule Changes**: The mention of a tech startup discussing new pay rules for H1B visas could impact the tech sector, as changes in immigration policies can affect the labor market and operational costs for tech companies.

3. **Bitcoin Trading**: The reference to Bitcoin trading suggests volatility in cryptocurrency markets, which can have ripple effects on broader financial markets, especially i